In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE

In [4]:
filepath2 = "/kaggle/input/scs-train/scs_train2.csv"
df2 = pd.read_csv(filepath2)
print(df2.shape)

(53214, 14)


In [5]:
x_train2 , x_test2 , y_train2 , y_test2 = train_test_split(
    df2.drop(labels=['TARGET'],axis=1),
    df2['TARGET'],
    test_size=0.3,
    random_state=0
)
print(x_train2.shape)
print(x_test2.shape)

(37249, 13)
(15965, 13)


In [9]:
sfs1 = SFS(RandomForestRegressor(n_jobs=4),
           k_features = 5,
           forward = True,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 3)
sfs1 = sfs1.fit(np.array(x_train2), y_train2)
x_train2.columns[list(sfs1.k_feature_idx_)]

[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:   15.0s finished

[2025-08-19 17:20:29] Features: 1/5 -- score: 0.021104260069296516[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   13.8s finished

[2025-08-19 17:20:43] Features: 2/5 -- score: 0.03706250480602813[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:   17.9s finished

[2025-08-19 17:21:01] Features: 3/5 -- score: 0.03265246596757428[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   17.4s finished

[2025-08-19 17:21:18] Features: 4/5 -- score: 0.029881382895844816[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   16.5s finished

[2025-08-19 17:21:35] Features: 5/5 -- score: 0.02804969621159718

Index(['var15', 'ind_var30', 'num_var4', 'num_var30', 'num_var42'], dtype='object')

In [10]:
sfs1 = SFS(RandomForestRegressor(n_jobs=4),
           k_features = 5,
           forward = False,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 3)
sfs1 = sfs1.fit(np.array(x_train2), y_train2)
x_train2.columns[list(sfs1.k_feature_idx_)]

[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:  1.1min finished

[2025-08-19 17:32:34] Features: 12/5 -- score: 0.019492352344251545[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   52.0s finished

[2025-08-19 17:33:26] Features: 11/5 -- score: 0.0198097362043587[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:   46.4s finished

[2025-08-19 17:34:12] Features: 10/5 -- score: 0.020079368839497607[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   40.8s finished

[2025-08-19 17:34:53] Features: 9/5 -- score: 0.019519903364813646[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   35.3s finished

[2025-08-19 17:35:28] Features: 8/5 -- score: 0.018865111306296756[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   30.7s finished

[2025-08-19 17:35:59] Features: 7/5 -- score: 0.019016667294706562[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   25.7s finished

[2025-08-19 17:36:25] Features: 6/5 -- score: 0.020081646402031545[Parallel(n_jobs=1)]: Done   6 out of   

Index(['var15', 'saldo_var30', 'var36', 'saldo_medio_var5_hace2',
       'saldo_medio_var5_ult1'],
      dtype='object')

In [7]:
lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500, solver='lbfgs'))
efs = EFS(estimator=lr,
          min_features=1,
          max_features=5,
          scoring='accuracy',
          print_progress=True,
          cv=5
)

efs = efs.fit(x_train2, y_train2)

print("Best score:", efs.best_score_)
print("Best feature indices:", efs.best_idx_)
print("Best feature names:", efs.best_feature_names_)

Features: 2379/2379

Best score: 0.9610996296066047
Best feature indices: (0,)
Best feature names: ('var15',)


In [6]:
lr = LogisticRegression(max_iter=500, solver='lbfgs')
rfe = RFE(estimator=lr, n_features_to_select=5)
pipeline = make_pipeline(StandardScaler(), rfe)
pipeline.fit(x_train2, y_train2)
selected_mask = pipeline.named_steps['rfe'].support_
rankings = pipeline.named_steps['rfe'].ranking_

print("Selected Features:", x_train2.columns[selected_mask].tolist())
print("Feature Ranking:", dict(zip(x_train2.columns, rankings)))

Selected Features: ['var15', 'ind_var30', 'saldo_var5', 'saldo_var30', 'saldo_medio_var5_ult3']
Feature Ranking: {'var15': 1, 'ind_var5': 4, 'ind_var30': 1, 'num_var4': 3, 'num_var30_0': 6, 'num_var30': 2, 'num_var42': 9, 'saldo_var5': 1, 'saldo_var30': 1, 'var36': 7, 'saldo_medio_var5_hace2': 5, 'saldo_medio_var5_ult1': 8, 'saldo_medio_var5_ult3': 1}
